In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(123)
from collections import OrderedDict as od
from itertools import chain

In [7]:
items_file = '../items.csv'
users_file = '../users.csv' 
trimmed_data = True
#directory where data will be stored
folder = '../prepare_data/test_dataset/'
test_interaction_file = folder + 'test_interaction.csv'
test_users_file = folder +'test_users.csv'
train_users_file = folder + 'train_users.csv'
#assigned gender file
assigned_users_file = folder + 'assigned_users.csv'
unbiased_test_interactions_file = folder + 'unbiased_test_interactions.csv'
biased_test_interactions_file = folder + 'biased_test_interactions.csv'
train_interactions_file = folder + 'train_interactions.csv'
train_negative_sampling_interactions_file = folder + 'negative_interactions.csv'
impression_file = '../impressions.csv'
jaccard_threshold = 0.2


In [3]:
from create_features import *
from process_xgb_data import *

In [4]:
user_key='user_id'
item_key='item_id'

In [5]:
from statistics import median,mean,mode
def load_file():
    items = pd.read_csv(items_file,sep='\t')
    items.fillna(-100,inplace=True)
    items = items.rename(columns={"id": "item_id"}, errors="raise")
    test_interaction = pd.read_csv(test_interaction_file)
    train_interactions = pd.read_csv(train_interactions_file)
    assigned_users = pd.read_csv(assigned_users_file)
    assigned_users.fillna(100,inplace=True)
    assigned_users=assigned_users.drop_duplicates(subset = [user_key])
    items=items.drop_duplicates(subset = [item_key])
    assigned_users = assigned_users.rename(columns={"id": "user_id"})
    return items,test_interaction,train_interactions,assigned_users

In [8]:
items,test_interactions,train_interactions,assigned_users = load_file()
items = item_process(items)
assigned_users = user_process(assigned_users)
negative_interactions = negative_sampling(train_interactions,user_key='user_id',item_key='item_id',timestamp=train_interactions.created_at.iloc[0])
negative_interactions = negative_interactions.drop('index',axis=1)
train_interactions = pd.concat([train_interactions,negative_interactions])
train_interactions['created_at'] = pd.to_datetime(train_interactions['created_at'], unit='s')
train_interactions['week'] = train_interactions.created_at.dt.strftime('%W')
positive_train_interactions = train_interactions[train_interactions['interaction_type']!=4]
#item_clicks = clicks(positive_train_interactions)
#click_ratio_df = click_ratio(positive_train_interactions)
#user_events = user_activity(train_interactions)
#item_id_max,user_id_max = click_date_max(train_interactions)
train_item_ids = train_interactions['item_id'].unique()
test_item_ids = test_interactions['item_id'].unique()
train_user_ids = train_interactions['user_id'].unique()
test_user_ids = test_interactions['user_id'].unique()
item_ids = np.unique(np.append(test_item_ids,train_item_ids))
items = items[items['item_id'].isin(item_ids)]

In [9]:
intu,intuser = Intu_Intusers(positive_train_interactions,assigned_users.set_index(user_key),items.set_index(item_key))

In [33]:
intu[intu.set_index('user_id').index.isin([83])]

,user_id,item_id,intu_career_level,intu_discipline_id,intu_industry_id,intu_region,intu_country,intu_latitude,intu_longitude,intu_mode_career_level,intu_mode_industry_id,intu_mode_discipline_id
0,83,"[470674, 550818, 649130, 1893735, 847916, 1510...","[3.0, 1.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, ...","[20, 16, 15, 15, 15, 8, 13, 15, 5, 13, 15, 16,...","[15, 9, 11, 11, 11, 15, 11, 11, 21, 11, 11, 9,...","[0, 1, 9, 0, 2, 1, 0, 0, 10, 0, 0, 1, 0, 7, 6,...","[de, de, de, non_dach, de, de, ch, ch, de, ch,...","[-100.0, 48.8, 52.3, 30.7, 50.3, 49.8, 47.5, 4...","[-100.0, 9.2, 10.2, -88.0, 8.9, 10.0, 8.7, 7.4...","[1.0, 3.0]","[15, 11]","[16, 15]"


In [11]:
items = item_postprocess(items,[intuser])
assigned_users = user_postprocess(assigned_users,[intu])


In [25]:
train_data =train_data.drop('item_id_y',axis=1).rename(columns={'item_id_x':'item_id'})

In [ ]:
train_data=train_interactions.merge(assigned_users,left_on=user_key,right_on=user_key)
train_data =train_data.merge(items,left_on=item_key,right_on=item_key,suffixes=('_user', '_item'))

In [26]:
train_data =train_data.merge(items,left_on=item_key,right_on=item_key,suffixes=('_user', '_item'))

In [27]:
event_feature(train_data)

0        [3.0, 1.0, 3.0, 1.0, 3.0, 3.0, 3.0, 3.0, 3.0, ...
1        [3.0, 1.0, 3.0, 1.0, 3.0, 3.0, 3.0, 3.0, 3.0, ...
2        [3.0, 1.0, 3.0, 1.0, 3.0, 3.0, 3.0, 3.0, 3.0, ...
3        [3.0, 3.0, 1.0, 1.0, 3.0, 3.0, 3.0, 3.0, 1.0, ...
4        [3.0, 1.0, 3.0, 1.0, 3.0, 3.0, 3.0, 3.0, 3.0, ...
                               ...                        
69932    [1.0, 3.0, 2.0, 3.0, 3.0, 1.0, 3.0, 3.0, 3.0, ...
69933    [1.0, 3.0, 2.0, 3.0, 3.0, 1.0, 3.0, 3.0, 3.0, ...
69934    [1.0, 3.0, 2.0, 3.0, 3.0, 1.0, 3.0, 3.0, 3.0, ...
69935    [1.0, 3.0, 2.0, 3.0, 3.0, 1.0, 3.0, 3.0, 3.0, ...
69936    [1.0, 3.0, 2.0, 3.0, 3.0, 1.0, 3.0, 3.0, 3.0, ...
Name: intu_career_level, Length: 69937, dtype: object
0        [9, 1, 2, 1, 9, 0, 9, 0, 9, 9, 3, 0, 4, 9, 9, ...
1        [9, 1, 2, 1, 9, 0, 9, 0, 9, 9, 3, 0, 4, 9, 9, ...
2        [9, 1, 2, 1, 9, 0, 9, 0, 9, 9, 3, 0, 4, 9, 9, ...
3        [0, 9, 0, 1, 0, 4, 1, 1, 9, 0, 1, 9, 9, 0, 9, ...
4        [9, 1, 2, 1, 9, 0, 9, 0, 9, 9, 3, 0, 4, 9, 9, ...
  

,Unnamed: 0,user_id,item_id,interaction_type,created_at_user,week,jobroles,career_level_user,discipline_id_user,industry_id_user,...,intu_discipline_id_item_contained,intu_country_item_contained,intuser_career_level_user_contained,intuser_region_user_contained,intuser_industry_id_user_contained,intuser_discipline_id_user_contained,intuser_country_user_contained,intuser_experience_years_experience_contained,intuser_edu_degree_contained,intuser_experience_years_in_current_contained
0,3791.0,865830,1427054,3,2015-10-30 06:29:04,43,1072229,3,20,1,...,0.176,0.882,0.75,1.0,0.75,0.75,1.0,0.75,1.0,0.75
1,3801.0,865830,1427054,1,2015-10-31 16:52:26,43,1072229,3,20,1,...,0.176,0.882,0.75,1.0,0.75,0.75,1.0,0.75,1.0,0.75
2,3805.0,865830,1427054,1,2015-10-30 06:28:17,43,1072229,3,20,1,...,0.176,0.882,0.75,1.0,0.75,0.75,1.0,0.75,1.0,0.75
3,5140281.0,2855502,1427054,1,2015-10-21 06:34:44,42,3731273,100,0,0,...,0.050,0.650,0.25,1.0,0.25,0.25,1.0,0.25,1.0,0.25
4,3792.0,865830,2312253,1,2015-11-03 14:14:03,44,1072229,3,20,1,...,0.294,0.882,0.50,1.0,0.50,0.50,1.0,0.50,1.0,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69932,8826621.0,2798291,1444038,1,2015-10-23 06:24:15,42,3580664,0,0,0,...,0.067,0.133,1.00,1.0,1.00,1.00,1.0,1.00,1.0,1.00
69933,8826616.0,2798291,1508101,1,2015-10-18 19:10:47,41,3580664,0,0,0,...,0.067,0.067,1.00,1.0,1.00,1.00,1.0,1.00,1.0,1.00
69934,8826619.0,2798291,1301190,1,2015-10-18 19:08:39,41,3580664,0,0,0,...,0.200,0.600,1.00,1.0,1.00,1.00,1.0,1.00,1.0,1.00
69935,8826623.0,2798291,1649684,1,2015-10-18 19:08:15,41,3580664,0,0,0,...,0.200,0.133,1.00,1.0,1.00,1.00,1.0,1.00,1.0,1.00


In [33]:
items,test_interactions,train_interactions,assigned_users = load_file()
items = item_process(items)
user_key='user_id'
item_key='item_id'
assigned_users=assigned_users.drop_duplicates('user_id')
assigned_users = user_process(assigned_users)
negative_interactions = negative_sampling(train_interactions,user_key='user_id',item_key='item_id',timestamp=train_interactions.created_at.iloc[0])
negative_interactions = negative_interactions.drop('index',axis=1)
train_interactions = pd.concat([train_interactions,negative_interactions])
train_interactions['created_at'] = pd.to_datetime(train_interactions['created_at'], unit='s')
train_interactions['week'] = train_interactions.created_at.dt.strftime('%W')
positive_train_interactions = train_interactions[train_interactions['interaction_type']!=4]
#item_clicks = clicks(positive_train_interactions)
#click_ratio_df = click_ratio(positive_train_interactions)
#user_events = user_activity(train_interactions)
#item_id_max,user_id_max = click_date_max(train_interactions)
train_item_ids = train_interactions['item_id'].unique()
test_item_ids = test_interactions['item_id'].unique()
train_user_ids = train_interactions['user_id'].unique()
test_user_ids = test_interactions['user_id'].unique()
item_ids = np.unique(np.append(test_item_ids,train_item_ids))
items = items[items['item_id'].isin(item_ids)]
intu,intuser = Intu_Intusers(positive_train_interactions,assigned_users.set_index('user_id'),items.set_index('item_id'))
user_item = intu[['user_id','item_id']].set_index('user_id')
intu = intu.drop('item_id',axis=1)

items = item_postprocess(items,[intuser,item_clicks,click_ratio_df,item_id_max])
assigned_users = user_postprocess(assigned_users,[intu,user_events,user_id_max])



train_items = items[items['item_id'].isin(train_item_ids)]
test_items = items[items['item_id'].isin(test_item_ids)]
user_item = intu_item_similarity(test_user_ids,test_items,intu.set_index('user_id'),user_item)
train_users = assigned_users[assigned_users['user_id'].isin(train_user_ids)]
test_users = assigned_users[assigned_users['user_id'].isin(test_user_ids)]
user_to_row = {user_id: i for i, user_id in enumerate(assigned_users['user_id'].unique())}
item_to_col = {item_id: i for i, item_id in enumerate(item_ids)}
jobrole_list = sum(assigned_users.jobrole_list, [])
tag_list = sum(items.tag_list, [])
title_list = sum(items.title_list, [])
item_tags = set(jobrole_list + tag_list +title_list)
tags_elements_dict =id_to_index(item_tags)
user_jobrole = test_users[['user_id','jobrole_list']].explode('jobrole_list')
user_jobrole['value'] =1
user_jobrole_sparse = sparcify(user_jobrole,'user_id','jobrole_list','value',user_to_row,tags_elements_dict)

train_item_title_sparse,train_item_tag_sparse = title_tag_sparse(train_items,item_to_col,tags_elements_dict)
test_item_title_sparse,test_item_tag_sparse = title_tag_sparse(test_items,item_to_col,tags_elements_dict)

testuser_i =[ user_to_row[id] for id in test_user_ids]
testuser_i = np.array(testuser_i)
sparse_collaborative_matrix = sparcify(positive_train_interactions.drop_duplicates(subset=['user_id','item_id']),'user_id','item_id','interaction_type',user_to_row,item_to_col)
testuser_mat= sparse_collaborative_matrix[testuser_i,:]
jaccard = jaccard_similarity(testuser_mat,sparse_collaborative_matrix)
user_item =jaccard_util(jaccard,test_user_ids,user_item,jaccard_threshold=0.5)

user_item = similarity_util(sparse_collaborative_matrix,test_user_ids,train_item_title_sparse,test_item_title_sparse,train_item_tag_sparse,test_item_tag_sparse,user_jobrole_sparse,user_item)
user_item = intu_item_similarity(test_user_ids,test_items,intu.set_index('user_id'),user_item)
test_data = user_item.reset_index().explode(['item_id'])
test_data = test_data.drop_duplicates(subset =['user_id','item_id'])
train_data,test_data = train_test_data(train_interactions,test_data,assigned_users,items,test_user_ids,test_item_ids,user_key='user_id',item_key='item_id')


/tmp/ipykernel_3828972/2900186027.py:204: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  matA_sum = scipy.sum(matA,axis=1)
/tmp/ipykernel_3828972/2900186027.py:257: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inter_df['jobrole_list'] =inter_df['jobrole_list'].apply(set)
/tmp/ipykernel_3828972/2900186027.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inter_df['title_list'] =inter_df['title_list'].apply(set)
/tmp/ipykernel_3828972/2900186027.py

In [73]:
from statistics import median,mean,mode
def load_file():
    items = pd.read_csv(items_file,sep='\t')
    items.fillna(-100,inplace=True)
    items = items.rename(columns={"id": "item_id"}, errors="raise")
    test_interaction = pd.read_csv(test_interaction_file)
    train_interactions = pd.read_csv(train_interactions_file)
    assigned_users = pd.read_csv(assigned_users_file)
    assigned_users.fillna(100,inplace=True)
    assigned_users=assigned_users.drop_duplicates(subset = [user_key])
    items=items.drop_duplicates(subset = [items_key])
    assigned_users = assigned_users.rename(columns={"id": "user_id"})
    return items,test_interaction,train_interactions,assigned_users

def missed_item_stats(recommended_list, groundtruth_list,test_user_ids):
  left_out_ids=[]  
  for user in test_user_ids:
    left_out_ids.append(len(set(groundtruth_list.loc[user]['item_id']) - set(groundtruth_list.loc[user]['item_id'])))
  print(f"mean {mean(left_out_ids)} median {median(left_out_ids)} and mode {mode(left_out_ids)}")

import random
def negative_sampling(train_interactions,user_key,item_key,sample_size=1,feedback_key='interaction_type',timestamp_key='created_at',timestamp=None):
    unique_users = train_interactions[user_key].drop_duplicates()
    unique_items = train_interactions[item_key].unique()
    non_interacted_pairs=unique_users.map(lambda user:  (user,set(random.sample(unique_items.tolist(),sample_size)) 
                                                         - set(train_interactions[train_interactions[user_key] == user][item_key])))
    non_interacted_df = pd.DataFrame(non_interacted_pairs.to_list(), columns=[user_key,item_key])
    non_interacted_df[item_key]= non_interacted_df[item_key].apply(list)
    non_interacted_df =non_interacted_df.explode(item_key).reset_index()
    non_interacted_df[feedback_key] =4
    non_interacted_df[timestamp_key] =timestamp
    return non_interacted_df
    
      
def train_test_data(train_data,test_data,users,items,test_user_ids,test_item_ids,user_key='user_id',item_key='item_id'):
    test_data = test_data.drop_duplicates(subset =['user_id','item_id'])
    test_data = test_data[test_data['user_id'].isin(test_user_ids)]
    items = intuser_process(items)
    train_data=train_data.merge(users,left_on=user_key,right_on=user_key)
    train_data =train_data.merge(items,left_on=item_key,right_on=item_key,suffixes=('_user', '_item'))
    test_data=test_data.merge(users,left_on=user_key,right_on=user_key)
    test_data=test_data.merge(items,left_on=item_key,right_on=item_key,suffixes=('_user', '_item'))
    train_data = content_similarity(train_data)
    test_data = content_similarity(test_data)
    train_data = last_click_activity(train_data)
    test_data = last_click_activity(test_data)
    train_data = event_feature(train_data)
    test_data = event_feature(test_data)
    return train_data,test_data
import numpy as np
from scipy.sparse import csr_matrix

def sparcify(df,row_name,col_name,value_name,user_to_row,item_to_col,row_shape=None,col_shape=None,):

    # Create a sparse matrix directly from data
    if not row_shape:
        row_shape = len(user_to_row)
    if not col_shape:    
        col_shape = len(item_to_col)

    # Create dictionaries to map user/item IDs to matrix indices
    #user_to_row = {user_id: i for i, user_id in enumerate(df[row_name].unique())}
    #item_to_col = {item_id: i for i, item_id in enumerate(df[col_name].unique())}
    # Initialize arrays to store rows, columns, and values for the sparse matrix
    rows = np.array([user_to_row[user_id] for user_id in df[row_name]])
    cols = np.array([item_to_col[item_id] for item_id in df[col_name]])
    values = np.repeat(1,len(rows))
    #df[value_name].values
    # Create the sparse matrix
    sparse_collaborative_matrix = csr_matrix((values, (rows, cols)), shape=(row_shape, col_shape))
    return sparse_collaborative_matrix

    
def id_to_index(df):
    item_to_col = {item_id: i for i, item_id in enumerate(df)}
    return item_to_col

def intuser_process(items):
    items['intuser_career_level'] = items['intuser_career_level'].apply(lambda x: [0] if  str(x) == 'nan' else x)
    items['intuser_discipline_id'] = items['intuser_discipline_id'].apply(lambda x: [0] if  str(x) == 'nan' else x)
    items['intuser_industry_id'] = items['intuser_industry_id'].apply(lambda x: [0] if  str(x) == 'nan' else x)
    items['intuser_region'] = items['intuser_region'].apply(lambda x: [0] if  str(x) == 'nan' else x)
    items['intuser_country'] = items['intuser_country'].apply(lambda x: [0] if  str(x) == 'nan' else x)
    items['intuser_experience_years_experience'] = items['intuser_experience_years_experience'].apply(lambda x: [0] if  str(x) == 'nan' else x)
    items['intuser_experience_years_in_current'] = items['intuser_experience_years_in_current'].apply(lambda x: [0] if  str(x) == 'nan' else x)
    items['intuser_edu_degree'] = items['intuser_edu_degree'].fillna(0).apply(lambda x: [0] if  str(x) == 'nan' else x)
    return items

def item_postprocess(items,merge_dfs,item_key='item_id'):
    for merge_df in merge_dfs:
       items=items.merge(merge_df,left_on=item_key,right_on=item_key,how='left')
    return(items)   

def user_postprocess(users,merge_dfs,user_key='user_id'):
    for merge_df in merge_dfs:
        users=users.merge(merge_df,left_on=user_key,right_on=user_key,how='left')
    return(users)   
def event_feature(df):
    columns_users = ['career_level_user','region_user','industry_id_user','discipline_id_user','country_user','experience_years_experience','edu_degree','experience_years_in_current']
   
    columns_item = ['career_level_item','region_item','industry_id_item','discipline_id_item','country_item']
    for column in columns_item:
        df['intu_'+column+'_contained'] = percentage_contained(df,'intu_'+column.replace('_item',''),column)
    for column in columns_users:
        df['intuser_'+column+'_contained'] = percentage_contained(df,'intuser_'+column.replace('_user',''),column)
    return df    
      

def clicks(df,user_key='user_id',item_key='item_id'):

    item_clicks = df.groupby(item_key)[user_key].count().reset_index(name = 'clicks')
    return item_clicks

def Intu_Intusers(df,users,items,user_key='user_id',item_key='item_id'):  

    Intusers = df.groupby([item_key])[user_key].apply(list).reset_index()
    Intu =df.groupby([user_key])[item_key].apply(list).reset_index()
    Intucolumns =['career_level', 'discipline_id', 'industry_id','region','country','latitude','longitude']
    Intusercolumns = [ 'career_level', 'discipline_id', 'industry_id','region','country',
       'experience_years_experience', 'experience_years_in_current','edu_degree']
    for column in Intucolumns:
           Intu['intu_'+ column] = Intu[item_key].apply(lambda items_list : [items.loc[item][column] for item in items_list])
    for column in Intusercolumns:
           Intusers['intuser_'+ column] = Intusers[user_key].apply(lambda users_list : [users.loc[user][column] for user in users_list]) 
    Intu['intu_mode_career_level'] = Intu['intu_career_level'].map(lambda x : sorted(set(x), key=x.count)[-2:])
    Intu['intu_mode_industry_id'] = Intu['intu_industry_id'].map(lambda x :  sorted(set(x), key=x.count)[-2:])
    Intu['intu_mode_discipline_id'] = Intu['intu_discipline_id'].map(lambda x :  sorted(set(x), key=x.count)[-2:])  
    Intusers.drop(user_key,axis=1,inplace=True) 
    return Intu,Intusers  

def item_process(items):
    items['title_list'] = items['title'].apply(lambda x: [int(ele ) for ele in str(x).split(",")])
    items['tag_list'] = items['tags'].apply(lambda x: [int(ele ) for ele in str(x).split(",")])
    #train_items =positive_items[positive_items['id'].isin(dropped_interactions_train[item_key].unique())]
    #test_items =positive_items[positive_items['id'].isin(dropped_interactions_test[item_key].unique())]
    #items =items[items[user_key].isin(train_interactions[item_key].unique())] 
    return items   

def intu_item_similarity(test_user_ids,test_items,Intu,user_items):
    for row_num,ele in enumerate(test_user_ids):
        intu_mode_career_level = Intu.loc[ele]['intu_mode_career_level']
        intu_mode_industry_id =  Intu.loc[ele]['intu_mode_industry_id']
        intu_mode_discipline_id =  Intu.loc[ele]['intu_mode_discipline_id']
        item_id_list = test_items[(test_items['career_level'].isin(intu_mode_career_level)) 
                                    & (test_items['industry_id'].isin(intu_mode_industry_id)) & 
                                    (test_items['discipline_id'].isin(intu_mode_discipline_id))]['item_id'].tolist()

        user_items['item_id'].loc[ele]=user_items['item_id'].loc[ele] + item_id_list   
    return user_items

def user_process(assigned_users):
     assigned_users['jobrole_list'] = assigned_users['jobroles'].apply(lambda x: [int(ele ) if len(str(x)) > 0 and str(x) != 'nan' else -100 for ele in str(x).split(",")])
     return assigned_users

def user_item_process():
        item_to_col = {item_id: i for i, item_id in enumerate(dropped_interactions_train['item_id'].unique())}
        user_items =Intu.copy()
        sparse_collaborative_matrix = sparcify(train_data,row_name,col_name,value_name,user_to_row,item_to_col)
        testuser_i =[ user_to_row[id] for id in test_user_ids]
        testuser_i = np.array(testuser_i)
        testuser_mat= sparse_collaborative_matrix[testuser_i,:]
        jaccard = jaccard_similarity(testuser_mat,sparse_collaborative_matrix)
        jaccard_util(jaccard,jaccard_threshold,test_user_ids,user_items)



def jaccard_util(jaccard,test_user_ids,user_items,jaccard_threshold):
    for row_num,ele in enumerate(test_user_ids):
        user_indexes  =np.argwhere(jaccard[row_num]>jaccard_threshold)[:,1]
        item_indexes = set(sparse_collaborative_matrix[user_indexes,:].nonzero()[1])
        item_id_list = np.take(item_ids,list(item_indexes))
        user_items['item_id'].loc[ele] = user_items['item_id'].loc[ele]+list(item_id_list)
    return user_items

def similarity_util(sparse_collaborative_matrix,test_user_ids,train_item_title_sparse,test_item_title_sparse,train_item_tag_sparse,test_item_tag_sparse,user_jobrole_sparse,user_items):
    ################################################
    title_title_similarity = intersection_similarity(train_item_title_sparse,test_item_title_sparse)
    title_tag_similarity = intersection_similarity(train_item_title_sparse,test_item_tag_sparse)
    tag_title_similarity = intersection_similarity(train_item_tag_sparse,test_item_title_sparse)
    tag_tag_similarity = intersection_similarity(train_item_tag_sparse,test_item_tag_sparse)
    jobrole_title_similarity = intersection_similarity(user_jobrole_sparse,test_item_tag_sparse)
    jobrole_tag_similarity = intersection_similarity(user_jobrole_sparse,test_item_title_sparse)
    for row_num,ele in enumerate(test_user_ids):
        testuser_i =user_to_row[ele]
        item_indexes = set(sparse_collaborative_matrix[testuser_i,:].nonzero()[1])
        titi_similar_item_indexes = list(chain(*[title_title_similarity[item_index,:].indices.tolist() for item_index in item_indexes]))
        tita_similar_item_indexes =list(chain(*[title_tag_similarity[item_index,:].indices[np.argsort(-title_tag_similarity[item_index,:].data)][:5].tolist() for item_index in item_indexes]))
        tati_similar_item_indexes =list(chain(*[tag_title_similarity[item_index,:].indices[np.argsort(-tag_title_similarity[item_index,:].data)][:5].tolist() for item_index in item_indexes]))
        tata_similar_item_indexes =list(chain(*[tag_tag_similarity[item_index,:].indices[np.argsort(-tag_tag_similarity[item_index,:].data)][:3].tolist() for item_index in item_indexes]))
        joti_similar_item_indexes =list(chain(*[jobrole_title_similarity[testuser_i,:].indices.tolist()]))
        jota_similar_item_indexes =list(chain(*[jobrole_tag_similarity[testuser_i,:].indices[np.argsort(-jobrole_tag_similarity[testuser_i,:].data)][:5].tolist()]))
        item_i = titi_similar_item_indexes + tita_similar_item_indexes + list(set(tati_similar_item_indexes).intersection(tata_similar_item_indexes).intersection(set(jota_similar_item_indexes))) + joti_similar_item_indexes 
        #item_i=list(chain(*item_i))
        item_id_list = np.unique(np.take(item_ids,item_i))
        item_id_list = item_id_list[np.in1d(item_id_list, test_item_ids, assume_unique=True)]
        user_items['item_id'].loc[ele]=user_items['item_id'].loc[ele]+list(item_id_list)
    return user_items

    
import scipy
from scipy.sparse import csr_matrix

def jaccard_similarity(matA,matB):

    intersection = matA.dot(matB.T)
    matA_sum = scipy.sum(matA,axis=1)
    matB_sum = matB.sum(axis=1)
    union = np.array(np.squeeze(matB_sum))[0] +matA_sum -intersection
    jaccard = intersection/union
    return jaccard

def click_ratio(df,user_key='user_id',item_key='item_id'):

    df = df.join(df.groupby('item_id')['created_at'].max(), on='item_id', rsuffix='_last_week_start')
    df = df.join(df.groupby('item_id')['created_at'].max(), on='item_id', rsuffix='_previous_last_week_start')
    df['created_at_last_week_start'] = df['created_at_last_week_start'] - pd.DateOffset(weeks=1)
    df['created_at_previous_last_week_start'] = df['created_at_previous_last_week_start']  - pd.DateOffset(weeks=2)

    last_week_data = df[(df['created_at'] > df['created_at_last_week_start'])]
    week_before_last_data = df[(df['created_at'] > df['created_at_previous_last_week_start']) & (df['created_at'] <= df['created_at_last_week_start'])]
    last_week_click_counts = last_week_data['item_id'].value_counts()
    week_before_last_click_counts = week_before_last_data['item_id'].value_counts()
    df =df.join(last_week_click_counts/week_before_last_click_counts, on='item_id',rsuffix='_click_ratio')
    df['item_id_click_ratio'] = df['item_id_click_ratio'].fillna(1)
    df = df[['item_id','item_id_click_ratio']].drop_duplicates(subset=['item_id'])
    return df


def user_activity(df,user_key='user_id',item_key='item_id'):

    user_events = df[[user_key,'item_id']].groupby(user_key)['item_id'].count().reset_index(name = 'user_activity')
    return user_events

def click_date_max(train):

    item_id_max = train.groupby(item_key)['created_at'].max().reset_index(name = 'item_id_max_time')
    user_id_max = train.groupby(user_key)['created_at'].max().reset_index(name = 'user_id_max_time')
    return item_id_max,user_id_max
    
def last_click_activity(df):
    import pandas as pd

# Load your interaction dataframe
    # interaction_df = pd.read_csv('interaction_data.csv')

    # Sort the dataframe by user_id and timestamp
   
    max_timestamp =  df['item_id_max_time'].max()
    df['max_click_time']=max_timestamp
    df['item_user_max_time'] = (df['item_id_max_time']-df['user_id_max_time']).dt.components['hours']
    df['item_id_max_time'] = (max_timestamp-df['item_id_max_time']).dt.components['hours']
    df['user_id_max_time'] = (max_timestamp-df['user_id_max_time']).dt.components['hours']
    return df

def content_similarity(df):

    df['career_diff'] = df['career_level_user'] -df['career_level_item']
    inter_df = df[['jobrole_list','title_list','tag_list']]
    inter_df['jobrole_list'] =inter_df['jobrole_list'].apply(set)
    inter_df['title_list'] =inter_df['title_list'].apply(set)
    inter_df['tag_list'] =inter_df['tag_list'].apply(set)
    df['job_title'] = inter_df.apply( lambda x: len(x['jobrole_list'].intersection(x['title_list'])),axis=1)
    df['job_tag'] = inter_df.apply( lambda x: len(x['jobrole_list'].intersection(x['tag_list'])),axis=1)
    return df

def intersection_similarity(matA,matB):
    intersection = matA.dot(matB.T)
    return intersection

def title_tag_sparse(df_items,item_to_col,tags_elements_dict,row_shape=None,col_shape=None):

    df_item_title = df_items[['item_id','title_list']].explode('title_list')
    df_item_title['value'] =1
    #user_to_rowtitle,item_to_coltitle =id_to_index(item_title,'id','title_list')
    df_item_title_sparse = sparcify(df_item_title,'item_id','title_list','value',item_to_col,tags_elements_dict,row_shape,col_shape)
    df_item_tag = train_items[['item_id','tag_list']].explode('tag_list')
    df_item_tag['value'] =1
    #user_to_rowtag,item_to_coltag =id_to_index(item_title,'id','tag_list')
    df_item_tag_sparse = sparcify(df_item_tag,'item_id','tag_list','value',item_to_col,tags_elements_dict,row_shape,col_shape)
    return df_item_title_sparse,df_item_tag_sparse

def percentage_contained(df,columnA,columnB):


  # we can compute these in-place, no need to create new DataFrame columns
  intersect = [elem.count(val) for elem, val in zip(df[columnA], df[columnB])]
  a_length = [len(elem) for elem in df[columnA]]

  return  [round(i/j,3) for i, j in zip(intersect, a_length)]


In [34]:
data =train_data[[ 'interaction_type', 'experience_n_entries_class',
       'experience_years_experience', 'experience_years_in_current',
       'edu_degree', 'latitude',
       'longitude','gender',
       'clicks',
       'item_id_click_ratio', 'item_id_max_time', 'career_diff', 'item_user_max_time',
       'intu_career_level_item_contained', 'intu_region_item_contained',
       'intu_industry_id_item_contained', 'intu_discipline_id_item_contained',
       'intu_country_item_contained', 'intuser_career_level_user_contained',
       'intuser_region_user_contained', 'intuser_industry_id_user_contained',
       'intuser_discipline_id_user_contained',
       'intuser_country_user_contained',
       'intuser_experience_years_experience_contained',
       'intuser_edu_degree_contained',
       'intuser_experience_years_in_current_contained']]

In [52]:
data_test=test_data[[ 'experience_n_entries_class',
       'experience_years_experience', 'experience_years_in_current',
       'edu_degree', 'latitude',
       'longitude','gender',
       'clicks',
       'item_id_click_ratio', 'item_id_max_time', 'career_diff', 'item_user_max_time',
       'intu_career_level_item_contained', 'intu_region_item_contained',
       'intu_industry_id_item_contained', 'intu_discipline_id_item_contained',
       'intu_country_item_contained', 'intuser_career_level_user_contained',
       'intuser_region_user_contained', 'intuser_industry_id_user_contained',
       'intuser_discipline_id_user_contained',
       'intuser_country_user_contained',
       'intuser_experience_years_experience_contained',
       'intuser_edu_degree_contained',
       'intuser_experience_years_in_current_contained']]

In [36]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [37]:
data['interaction_type'] = ((data['interaction_type']==1) | (data['interaction_type']==2) | (data['interaction_type']==3)).astype(int)
X_train, X_test, y_train, y_test = train_test_split(data.drop('interaction_type',axis=1), data['interaction_type'], test_size=.3)
bst = XGBClassifier(n_estimators=100,n_jobs=100, max_depth=6, learning_rate=0.1,gamma=1,min_child_weight=4,gpu_id=6,tree_method='gpu_hist',objective='binary:logistic')
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 77.78%


/tmp/ipykernel_3828972/2447738504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['interaction_type'] = ((data['interaction_type']==1) | (data['interaction_type']==2) | (data['interaction_type']==3)).astype(int)


In [49]:
data_test.columns.shape

(25,)

In [54]:
test_data['y_pred'] = bst.predict_proba(data_test)[:,1]
sorted_items = test_data[['user_id','item_id','y_pred']].sort_values(by=['y_pred'], ascending=[False]).groupby('user_id').head(30)
recommended_users = sorted_items.groupby('user_id')['item_id'].agg(list).reset_index()
Y_labels =test_interactions[['user_id','item_id']].groupby('user_id')['item_id'].agg(list).reset_index().set_index('user_id')

In [74]:
missed_item_stats(recommended_users.set_index('user_id'), Y_labels,test_user_ids)

mean 0 median 0.0 and mode 0


In [ ]:
from  metrics import challenge_score,compute_metrics,totalRelevant,GCE
print(compute_metrics(recommended_users.set_index('user_id'),Y_labels))